In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pdfplumber
import datetime

# Analysis of Results of the 2019 ARA National Championships Presented by AMSOIL #
*Darren McGlinchey*

The [American Rally Association (ARA)](https://www.americanrallyassociation.org/) is the top stage rally competition in the United States. I've been following the success of the [Subaru Motorsports USA](https://www.subaru.com/motorsports) team via the popular YouTube show "Launch Control". The team has dominated the rally scene in the US for the last decade, particularly with the Driver/Co-Driver pair David Higgens & Craig Drew. I was curious if the dominance of Subaru in the top Open 4 wheel drive (O4WD) class trickled down. How popular are Subaru's throughout the classes in which they can compete? How successful are they in their classes?

I chose to look into answering these questions beginning with the 2019 season. I chose the 2019 season for a few reasons. First, it is recent and wasn't effected by the Covid pandemic like the 2020 season. And secondly, it was the last season in which Higgens & Drew drove for Subaru Motorsports USA.

## Getting & Cleaning the Data ##

In order to answer the above questions I'm looking for the following data:
* National results for each race in the 2019 season (I'll ignore regional results for now)
* Primarily interested in overall results rather than individual stage times
* Should contain overall position, driver/co-driver, car, class, total time, and position in class.

I was unable to find a compiled and available dataset containing results of the 2019 season. Some, but not all, of the 2019 results can be found on the [ARA](https://www.americanrallyassociation.org/) website, however not in an easily scrapable format. While compiling the results by hand would not be a huge undertaking given the number of races (9) and modest number of competitors per race (14-49), thankfully there was another way. 

The website [ewrc-results.com](https://www.ewrc-results.com/) is a website by rally fans attempting to compile a public database of world rally events and results. They appear to have complete listings and results for each race in the season in a more scraping-friendly format. I'll start there.

Let's start by getting the results for each event individually. There were 9 events in the [2019 ARA season](https://www.ewrc-results.com/season/2019/994-ara/):
1. [Sno*Drift Rally](https://www.ewrc-results.com/final/56054-snodrift-rally-2019/) (SNO), 25-26 Jan 2019, Atlanta Michigan
2. [Rally in the 100 Acre Wood](https://www.ewrc-results.com/final/56055-rally-in-the-100-acre-wood-2019/) (RAW), 15-16 Mar 2019, Salem, Missouri
3. [DirtFish Olympus Rally](https://www.ewrc-results.com/final/56056-dirtfish-olympus-rally-2019/) (DOR), 27-28 Apr 2019, Shelton, Washington
4. [Oregon Trail Rally](https://www.ewrc-results.com/final/56057-oregon-trail-rally-2019/) (OTR), 31 May - 2 Jun 2019, Portland, Oregon
5. [Idaho Rally International](https://www.ewrc-results.com/final/56058-idaho-rally-international-2019/) (IR), 22-23 Jun 2019, Idaho City, Idaho
6. [New England Forest Rally](https://www.ewrc-results.com/final/56059-new-england-forest-rally-2019/) (NEFR), 19-20 Jul 2019, Bethel, Maine
7. [Muscatell Ojibwe Forests Rally](https://www.ewrc-results.com/final/56060-muscatell-ojibwe-forests-rally-2019/) (OFR), 22-24 Aug 2019, Detroit Lakes, Minnesota
8. [Waste Management Susquehannock Trail Performance Rally](https://www.ewrc-results.com/final/56061-waste-management-susquehannock-trail-performance-rally-2019/) (STPR), 13-14 Sep 2019, Wellsboro, Pennsylvania
9. [Lake Superior Performance Rally](https://www.ewrc-results.com/final/56062-lake-superior-performance-rally-2019/) (LSPR), 17-19 Oct 2019, Houghton, Michigan


In [2]:
# Setup the necessary urls
ara_2019_season_url = 'https://www.ewrc-results.com/season/2019/994-ara/'
ara_2019_rally_urls = ['https://www.ewrc-results.com/final/56054-snodrift-rally-2019/',
                       'https://www.ewrc-results.com/final/56055-rally-in-the-100-acre-wood-2019/',
                       'https://www.ewrc-results.com/final/56056-dirtfish-olympus-rally-2019/',
                       'https://www.ewrc-results.com/final/56057-oregon-trail-rally-2019/',
                       'https://www.ewrc-results.com/final/56058-idaho-rally-international-2019/',
                       'https://www.ewrc-results.com/final/56059-new-england-forest-rally-2019/',
                       'https://www.ewrc-results.com/final/56060-muscatell-ojibwe-forests-rally-2019/',
                       'https://www.ewrc-results.com/final/56061-waste-management-susquehannock-trail-performance-rally-2019/',
                       'https://www.ewrc-results.com/final/56062-lake-superior-performance-rally-2019/']

Before getting all the results, let's make a data frame containing the event information.

In [3]:
ara_2019_rally_abbrev = ['SNO', 'RAW', 'DOR', 'OTR', 'IR', 'NEFR', 'OFR', 'STPR', 'LSPR']
ara_2019_rally_names = ['Sno*Drift Rally', 'Rally in the 100 Acre Wood', 'DirtFish Olympus Rally',
                       'Oregon Trail Rally', 'Idaho Rally International', 'New England Forest Rally',
                       'Muscatell Ojibwe Forests Rally', 'Waste Management Susquehannock Trail Performance Rally',
                       'Lake Superior Performance Rally']
ara_2019_rally_start_date = [datetime.date(2019,1,25), datetime.date(2019, 3, 15), datetime.date(2019, 4, 27),
                            datetime.date(2019, 5, 31), datetime.date(2019, 6, 22), datetime.date(2019, 7, 19),
                            datetime.date(2019, 8, 22), datetime.date(2019, 9, 13), datetime.date(2019, 10, 17)]
ara_2019_rally_end_date = [datetime.date(2019, 1, 26), datetime.date(2019, 3, 16), datetime.date(2019, 4, 28),
                          datetime.date(2019, 6, 2), datetime.date(2019, 6, 23), datetime.date(2019, 7, 20),
                          datetime.date(2019, 8, 24), datetime.date(2019, 9, 14), datetime.date(2019, 10, 19)]
ara_2019_rally_loc = ['Atlanta, Michigan', 'Salem, Missouri', 'Shelton, Washington',
                     'Portland, Oregon', 'Idaho City, Idaho', 'Bethal, Maine', 
                     'Detroit Lakes, Minnesota', 'Wellsboro, Pennsylvania', 'Houghton, Michigan']
ara_2019_rally_state = ['MI', 'MO', 'WA', 'OR', 'ID', 'ME', 'MN', 'PA', 'MI']

In [4]:
ara_2019_df = pd.DataFrame({'abbrev':ara_2019_rally_abbrev,
                            'name':ara_2019_rally_names,
                            'start_date':ara_2019_rally_start_date,
                            'end_date':ara_2019_rally_end_date,
                            'location':ara_2019_rally_loc,
                            'state':ara_2019_rally_state})
ara_2019_df

,abbrev,name,start_date,end_date,location,state
0,SNO,Sno*Drift Rally,2019-01-25,2019-01-26,"Atlanta, Michigan",MI
1,RAW,Rally in the 100 Acre Wood,2019-03-15,2019-03-16,"Salem, Missouri",MO
2,DOR,DirtFish Olympus Rally,2019-04-27,2019-04-28,"Shelton, Washington",WA
3,OTR,Oregon Trail Rally,2019-05-31,2019-06-02,"Portland, Oregon",OR
4,IR,Idaho Rally International,2019-06-22,2019-06-23,"Idaho City, Idaho",ID
5,NEFR,New England Forest Rally,2019-07-19,2019-07-20,"Bethal, Maine",ME
6,OFR,Muscatell Ojibwe Forests Rally,2019-08-22,2019-08-24,"Detroit Lakes, Minnesota",MN
7,STPR,Waste Management Susquehannock Trail Performan...,2019-09-13,2019-09-14,"Wellsboro, Pennsylvania",PA
8,LSPR,Lake Superior Performance Rally,2019-10-17,2019-10-19,"Houghton, Michigan",MI


We can also try to see what we can get from the season overview table at ewrc-results.com

In [5]:
season_tables = pd.read_html(ara_2019_season_url)
print('season tables: {}'.format(len(season_tables)))
for i, table in enumerate(season_tables):
    print(' table {}, shape: {}'.format(i, table.shape))

season tables: 22
 table 0, shape: (3, 7)
 table 1, shape: (3, 7)
 table 2, shape: (3, 7)
 table 3, shape: (3, 7)
 table 4, shape: (3, 7)
 table 5, shape: (3, 7)
 table 6, shape: (3, 7)
 table 7, shape: (3, 7)
 table 8, shape: (3, 7)
 table 9, shape: (3, 7)
 table 10, shape: (3, 7)
 table 11, shape: (3, 7)
 table 12, shape: (3, 7)
 table 13, shape: (3, 7)
 table 14, shape: (3, 7)
 table 15, shape: (3, 7)
 table 16, shape: (3, 7)
 table 17, shape: (3, 7)
 table 18, shape: (53, 13)
 table 19, shape: (5, 4)
 table 20, shape: (8, 4)
 table 21, shape: (6, 4)


In [6]:
season_tables[0]

,0,1,2,3,4,5,6
0,1.0,NaN,Fetela Piotr - Jozwiak Dominik,NaN,Ford Fiesta Proto,Fetela Rally Team,2:20:04.5
1,2.0,NaN,Steely Cameron - Osborn Preston,NaN,Subaru Impreza WRX STi,O.D.D. Racing,2:24:01.5
2,3.0,NaN,Nease Travis - James Matthew,NaN,Subaru WRX STI,Hi Camp Racing,2:31:51.9


In [7]:
season_tables[1]

,0,1,2,3,4,5,6
0,1.0,NaN,Fetela Piotr - Jozwiak Dominik,NaN,Ford Fiesta Proto,Fetela Rally Team,2:20:04.5
1,2.0,NaN,Steely Cameron - Osborn Preston,NaN,Subaru Impreza WRX STi,O.D.D. Racing,2:24:01.5
2,3.0,NaN,Nease Travis - James Matthew,NaN,Subaru WRX STI,Hi Camp Racing,2:31:51.9


In [8]:
season_tables[2]
type(season_tables[2])

pandas.core.frame.DataFrame

Based on the website, I would have expected 9 tables (1 for each race) with the top three finishers followed by a few tables of additional statistics. Those results are certainly in there, but let's clean it up. It looks like the results tables got duplicated, and there are some `NaN` columns where the flag/logo's go.

In [9]:
season_dfs = []
for i in range(0, 18, 2):    
    season_tables[i].columns = ['pos','A','driverco','B','car','team','time']
    driver = season_tables[i].driverco.apply(lambda row: row.split(' - ')[0])
    codriver = season_tables[i].driverco.apply(lambda row: row.split(' - ')[1])
    df = season_tables[i].assign(driver=driver.values)
    df = df.assign(codriver=codriver.values)
    df = df.drop(['A','B','driverco'], axis=1)
    season_dfs.append(df)
print('N races: {}'.format(len(season_dfs)))
season_dfs[0]    

N races: 9


,pos,car,team,time,driver,codriver
0,1.0,Ford Fiesta Proto,Fetela Rally Team,2:20:04.5,Fetela Piotr,Jozwiak Dominik
1,2.0,Subaru Impreza WRX STi,O.D.D. Racing,2:24:01.5,Steely Cameron,Osborn Preston
2,3.0,Subaru WRX STI,Hi Camp Racing,2:31:51.9,Nease Travis,James Matthew


That looks better. 

Now lets get tables of full results for each of the races. First, let's take a quick look at some results from the first race of the season before we try grabbing all the results.

In [10]:
test_tabs = pd.read_html(ara_2019_rally_urls[0])
print(len(test_tabs))
for i, table in enumerate(test_tabs):
    print('table {} shape: {}'.format(i, table.shape))
print('table[0][:,0]:\n{}'.format(test_tabs[0].iloc[0]))
print('table[1][:,0]:\n{}'.format(test_tabs[1].iloc[0]))

2
table 0 shape: (16, 9)
table 1 shape: (2, 7)
table[0][:,0]:
0                                               1
1                                             #94
2                                             NaN
3                  Fetela Piotr - Jozwiak Dominik
4    Ford Fiesta Proto [FRT 94] Fetela Rally Team
5                                            O4WD
6                                   2:20:04.50:10
7                                             NaN
8                                            80.8
Name: 0, dtype: object
table[1][:,0]:
0                                             SS8
1                                             #97
2                                             NaN
3                     McKenna Barry - Jordan Leon
4    Ford Fiesta R5 [JEA-4540]McKenna Motorsports
5                                            O4WD
6                                      Mechanical
Name: 0, dtype: object


That makes sense. We get 2 tables. The first should be the full results, with the second detailing the retirements. Let's make a function to merge those together into a commmon data frame with only the columns we care about. We need:
* The teams class
* The teams overall finish and time
* The driver & co-driver
* The car's manufacturer (or make)
* The car's number (can be used for easy tracking later)

In [11]:
def get_event(url):
    '''Retrieve and format ARA event tables from website.
    
    Read HTML from "url" containing ARA event tables from
    ewrc-results.com and format into a usable data frame.
    
    Arguments:
        url (str): web page url containing results tables
        
    Returns:
        Pandas Dataframe containing event results
    '''
    web_tables = pd.read_html(url)
    if len(web_tables) != 2:
        print('Found {} tables, exxpected 2 from {}'.format(len(web_tables), url))
        return None
    # --- set the column names based on the web page structure
    web_tables[0].columns = ['pos','carnum','A','driverco','carteam','class','jumbledtime','tdiff','unknown']
    web_tables[1].columns = ['stage','carnum','A','driverco','carteam','class','reasonexit']
    # --- format the first table (entries who finished the event)
    res_df = web_tables[0].drop(['A','tdiff','unknown'], axis=1)
    # split the driver & co-driver
    drivers = res_df.driverco.apply(lambda row: row.split(' - ')[0])
    codrivers = res_df.driverco.apply(lambda row: row.split(' - ')[1])
    res_df = res_df.assign(driver=drivers.values)
    res_df = res_df.assign(codriver=codrivers.values)
    # reformat the times, dropping the jumbled time time to next place that gets appended
    times = res_df.jumbledtime.apply(lambda row: row[:row.find('.')+2])
    res_df = res_df.assign(time=times.values)
    # get the car manufacturer (make). Assumed to be the first word in 'carteam' column
    makes = res_df.carteam.apply(lambda row: row.split(' ')[0])
    res_df = res_df.assign(make=makes.values)
    res_df = res_df.drop(['driverco','jumbledtime'], axis=1)
    # --- format the second table (retired entries) - perform same actions as on first table
    ret_df = web_tables[1].drop(['stage','A','reasonexit'], axis=1)
    drivers = ret_df.driverco.apply(lambda row: row.split(' - ')[0])
    codrivers = ret_df.driverco.apply(lambda row: row.split(' - ')[1])
    ret_df = ret_df.assign(driver=drivers.values)
    ret_df = ret_df.assign(codriver=codrivers.values)
    # for DNF's make the position NaN
    poss = ret_df.carnum.apply(lambda row: np.nan)
    ret_df = ret_df.assign(pos=poss.values)
    # for DNF's make the time NaN
    times = ret_df.carnum.apply(lambda row: np.nan)
    ret_df = ret_df.assign(time=times.values)
    makes = ret_df.carteam.apply(lambda row: row.split(' ')[0])
    ret_df = ret_df.assign(make=makes.values)
    ret_df = ret_df.drop(['driverco'], axis=1)
    # combine the two tables
    df = res_df.append(ret_df)
    # for this work, only care about 4WD classes (Subaru's are 4WD!)
    # df = df[df['class'].str.contains('4WD')]
    # done
    return df

Let's grab tables for each event (we'll merge them together later)

In [12]:
event_dfs = [get_event(url) for url in ara_2019_rally_urls]
event_dfs[0]

,pos,carnum,carteam,class,driver,codriver,time,make
0,1.0,#94,Ford Fiesta Proto [FRT 94] Fetela Rally Team,O4WD,Fetela Piotr,Jozwiak Dominik,2:20:04.5,Ford
1,2.0,#824,Subaru Impreza WRX STi [410 0598] O.D.D. Racing,L4WD,Steely Cameron,Osborn Preston,2:24:01.5,Subaru
2,3.0,#81,Subaru WRX STIHi Camp Racing,L4WD,Nease Travis,James Matthew,2:31:51.9,Subaru
3,4.0,#845,Honda CivicTeam Punishment Racing,O2WD,MacDonald Shawn,Cannis Jonathan,2:41:17.9,Honda
4,5.0,#777,Subaru ImprezaHeavy Metal,NA4WD,Kramer Jonathan,Smith Jason,2:43:54.5,Subaru
5,6.0,#123,Subaru Impreza RSMBP Motorsports,NA4WD,Engle Michael Jr,Engle Lauren,2:45:32.9,Subaru
6,7.0,#386,Mitsubishi Lancer [CMZ-734] Morris Motorsports,O2WD,Morris Bradley,Nagy Douglas,2:45:52.4,Mitsubishi
7,8.0,#98,Subaru Impreza WRX STi RABMG Racing,L4WD,Bardha Ele,Roshea Corrina,2:53:51.8,Subaru
8,9.0,#815,Subaru Impreza WRX STiToasted Racing,L4WD,Whitebread Zachary,Carr Cameron,2:56:05.6,Subaru
9,10.0,#686,Subaru Impreza RS [ABM-4363] Leadfoot Locher R...,NA4WD,Locher Jordan,Addison Thomas,3:00:42.2,Subaru


In [13]:
print(len(event_dfs))

9


Looks good, before merging all the events together, let's get some summary information and add it to the season data.

In [14]:
rally_entries = [df.shape[0] for df in event_dfs]
rally_finishers = [df.dropna().shape[0] for df in event_dfs]
rally_4wd = [df[df['class'].str.contains('4WD')].shape[0] for df in event_dfs]
rally_subarus = [df[df['make'].str.contains('Subaru')].shape[0] for df in event_dfs]

In [15]:
ara_2019_df = ara_2019_df.assign(entries=rally_entries,
                                finishers=rally_finishers,
                                awd_entries=rally_4wd,
                                subaru_entries=rally_subarus)
ara_2019_df

,abbrev,name,start_date,end_date,location,state,entries,finishers,awd_entries,subaru_entries
0,SNO,Sno*Drift Rally,2019-01-25,2019-01-26,"Atlanta, Michigan",MI,18,16,13,11
1,RAW,Rally in the 100 Acre Wood,2019-03-15,2019-03-16,"Salem, Missouri",MO,49,31,32,24
2,DOR,DirtFish Olympus Rally,2019-04-27,2019-04-28,"Shelton, Washington",WA,24,18,15,12
3,OTR,Oregon Trail Rally,2019-05-31,2019-06-02,"Portland, Oregon",OR,31,25,18,14
4,IR,Idaho Rally International,2019-06-22,2019-06-23,"Idaho City, Idaho",ID,14,11,10,9
5,NEFR,New England Forest Rally,2019-07-19,2019-07-20,"Bethal, Maine",ME,30,20,22,17
6,OFR,Muscatell Ojibwe Forests Rally,2019-08-22,2019-08-24,"Detroit Lakes, Minnesota",MN,29,21,17,15
7,STPR,Waste Management Susquehannock Trail Performan...,2019-09-13,2019-09-14,"Wellsboro, Pennsylvania",PA,25,21,17,13
8,LSPR,Lake Superior Performance Rally,2019-10-17,2019-10-19,"Houghton, Michigan",MI,29,17,19,14


Now let's merge all the individual event information into a single dataframe, adding the event abbreviation.

In [19]:
ara_2019_results = event_dfs[0].assign(event=ara_2019_rally_abbrev[0])
for i in range(1, len(event_dfs)):
    ara_2019_results = ara_2019_results.append(event_dfs[i].assign(event=ara_2019_rally_abbrev[i]))
# only keep entries in 4WD categories
ara_2019_results = ara_2019_results[ara_2019_results['class'].str.contains('4WD')]
ara_2019_results.shape

(163, 9)

In [20]:
ara_2019_results.reset_index(drop=True)
ara_2019_results

,pos,carnum,carteam,class,driver,codriver,time,make,event
0,1.0,#94,Ford Fiesta Proto [FRT 94] Fetela Rally Team,O4WD,Fetela Piotr,Jozwiak Dominik,2:20:04.5,Ford,SNO
1,2.0,#824,Subaru Impreza WRX STi [410 0598] O.D.D. Racing,L4WD,Steely Cameron,Osborn Preston,2:24:01.5,Subaru,SNO
2,3.0,#81,Subaru WRX STIHi Camp Racing,L4WD,Nease Travis,James Matthew,2:31:51.9,Subaru,SNO
4,5.0,#777,Subaru ImprezaHeavy Metal,NA4WD,Kramer Jonathan,Smith Jason,2:43:54.5,Subaru,SNO
5,6.0,#123,Subaru Impreza RSMBP Motorsports,NA4WD,Engle Michael Jr,Engle Lauren,2:45:32.9,Subaru,SNO
...,...,...,...,...,...,...,...,...,...
2,NaN,#74,Subaru Impreza S7 WRC '01 [X27 SRT]Ramana Lage...,O4WD,Lagemann Ramana,Richard Nathalie,NaN,Subaru,LSPR
3,NaN,#121,Subaru Impreza WRX Wagon,NA4WD,Usher Nathan,Usher Marianna,NaN,Subaru,LSPR
6,NaN,#474,Subaru Impreza RS,NA4WD,Hueser Jeff,Stevens Dylan,NaN,Subaru,LSPR
7,NaN,#489,Subaru WRX STI,L4WD,Dickinson Matthew,Piker Daniel,NaN,Subaru,LSPR


Now we have both the season and event information in nicely formatted dataframes. Let's save them to csv for use in visualizations.

In [22]:
ara_2019_df.to_csv('ARA_2019_season_info.csv', index=False)
ara_2019_results.to_csv('ARA_2019_event_info.csv', index=False)